<a href="https://colab.research.google.com/github/LennyRBriones/embeddings/blob/main/Embeddings_search_proyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers
!pip install pinecone-client
!pip install gradio
!pip install typing-extensions --upgrade

  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from ast import literal_eval

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Models/25k-imdb-movie-dataset_6876f742-4304-4e96-b695-41e146a046c3.csv")

In [4]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Plot Kyeword,Director,Top 5 Casts,Writer,year,path
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,"['fighter jet', 'sequel', 'u.s. navy', 'fighte...",Joseph Kosinski,"['Jack Epps Jr.', 'Peter Craig', 'Tom Cruise',...",Jim Cash,-2022,/title/tt1745960/
1,Jurassic World Dominion,2 hours 27 minutes,6,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,"['dinosaur', 'jurassic park', 'tyrannosaurus r...",Colin Trevorrow,"['Colin Trevorrow', 'Derek Connolly', 'Chris P...",Emily Carmichael,-2022,/title/tt8041270/
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,"['pilot', 'male camaraderie', 'u.s. navy', 'gr...",Tony Scott,"['Jack Epps Jr.', 'Ehud Yonay', 'Tom Cruise', ...",Jim Cash,-1986,/title/tt0092099/


Clean Data

In [5]:
def concatenate_list(list):
    list = literal_eval(list)
    return "".join(list)

In [6]:
def string_to_list(list):
    lista = literal_eval(list)
    return list

In [7]:
df = df.fillna(" ")
df["Keywords"] = df["Plot Kyeword"].apply(concatenate_list)
df["Stars"] = df["Top 5 Casts"].apply(concatenate_list)
df["Generes"] = df["Generes"].apply(string_to_list)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce").fillna(0).astype("float")

In [8]:
unique_generes = df["Generes"].explode().unique()
unique_generes

array(["['Action', 'Drama']", "['Action', 'Adventure', 'Sci-Fi']",
       "['Animation', 'Action', 'Adventure']",
       "['Action', 'Crime', 'Drama']",
       "['Action', 'Adventure', 'Comedy']",
       "['Action', 'Adventure', 'Drama']",
       "['Action', 'Sci-Fi', 'Thriller']",
       "['Action', 'Fantasy', 'Sci-Fi']",
       "['Action', 'Adventure', 'Fantasy']",
       "['Action', 'Comedy', 'Crime']",
       "['Action', 'Adventure', 'Thriller']", "['Action', 'Thriller']",
       "['Action', 'Adventure', 'Horror']", "['Action', 'Adventure']",
       "['Action', 'Drama', 'Horror']",
       "['Adventure', 'Drama', 'Horror']",
       "['Comedy', 'Horror', 'Thriller']",
       "['Action', 'Drama', 'Thriller']",
       "['Action', 'Drama', 'History']", "['Action', 'Sci-Fi']",
       "['Drama']", "['Action', 'Drama', 'Mystery']",
       "['Action', 'Biography', 'Drama']",
       "['Action', 'Crime', 'Thriller']",
       "['Action', 'Comedy', 'Fantasy']",
       "['Action', 'Adventure', '

In [9]:
df.drop(["Plot Kyeword", "Top 5 Casts"],axis=1, inplace=True)

In [10]:
df["text"] = df.apply(lambda x : str(x["Overview"])+ " " +x["Keywords"]+ " "+ x["Stars"], axis=1)

In [11]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jetsequelu.s. navyfighter aircraftpilo...,Jack Epps Jr.Peter CraigTom CruiseJennifer Con...,After more than thirty years of service as one...
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaurjurassic parktyrannosaurus rexvelocira...,Colin TrevorrowDerek ConnollyChris PrattBryce ...,Four years after the destruction of Isla Nubla...
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilotmale camaraderieu.s. navygrumman f 14 tom...,Jack Epps Jr.Ehud YonayTom CruiseTim RobbinsKe...,As students at the United States Navy's elite ...


Moving the cleaned data set to a vectorial space

In [12]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [13]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/382 [00:00<?, ?it/s]

In [14]:
df["embeddings"] = embeddings.tolist()
df["ids"] = df.index
df["ids"] = df["ids"].astype("str")

In [15]:
df.head(3)

,movie title,Run Time,Rating,User Rating,Generes,Overview,Director,Writer,year,path,Keywords,Stars,text,embeddings,ids
0,Top Gun: Maverick,"$170,000,000 (estimated)",8.6,187K,"['Action', 'Drama']",After more than thirty years of service as one...,Joseph Kosinski,Jim Cash,-2022,/title/tt1745960/,fighter jetsequelu.s. navyfighter aircraftpilo...,Jack Epps Jr.Peter CraigTom CruiseJennifer Con...,After more than thirty years of service as one...,"[-0.04837854206562042, -0.003710306016728282, ...",0
1,Jurassic World Dominion,2 hours 27 minutes,6.0,56K,"['Action', 'Adventure', 'Sci-Fi']",Four years after the destruction of Isla Nubla...,Colin Trevorrow,Emily Carmichael,-2022,/title/tt8041270/,dinosaurjurassic parktyrannosaurus rexvelocira...,Colin TrevorrowDerek ConnollyChris PrattBryce ...,Four years after the destruction of Isla Nubla...,"[-0.030405916273593903, -0.018286526203155518,...",1
2,Top Gun,"$15,000,000 (estimated)",6.9,380K,"['Action', 'Drama']",As students at the United States Navy's elite ...,Tony Scott,Jim Cash,-1986,/title/tt0092099/,pilotmale camaraderieu.s. navygrumman f 14 tom...,Jack Epps Jr.Ehud YonayTom CruiseTim RobbinsKe...,As students at the United States Navy's elite ...,"[0.026411941275000572, 0.05586894601583481, -0...",2


In [16]:
import pinecone
from getpass import getpass

In [17]:
pinecone_api = getpass("Enter the value...")

Enter the value...··········


In [18]:
pinecone.init(api_key= pinecone_api, environment="gcp-starter")

In [19]:
dimension_embeddings= len(df["embeddings"][0])
index_name = "movies-embeddings"
all_index = pinecone.list_indexes()
if index_name in all_index:
  index = pinecone.Index(index_name)
else:
  pinecone.create_index(index_name, dimension= dimension_embeddings, metric="cosine")
  index = pinecone.Index(index_name)


In [20]:
index

In [21]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text','path'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb,metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/382 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.24402,
 'namespaces': {'': {'vector_count': 24402}},
 'total_vector_count': 24402}

In [22]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.24402,
 'namespaces': {'': {'vector_count': 24402}},
 'total_vector_count': 24402}

In [23]:
def search(query, genre, rating, top_k):
  query_vector = model.encode(query).tolist()

  if rating:
      filter_rating = rating
  else:
      filter_rating = 0
  if genre:
      conditions = {
          "Generes" : {
              "$in" : [genre]

          },
          "Rating" : {
              "$gte": filter_rating
          }
      }
  else:
      conditions = {
          "Rating": {
              "$gte": filter_rating
          }
      }

  responses = index.query(
      vector = query_vector,
      top_k = top_k,
      include_metadata=True,
      filkter = conditions
  )
  response_data = []
  for response in responses["matches"]:
    response_data.append(
        {
            "Title" : response["metadata"]["movie title"],
            "Overview" : response["metadata"]["Overview"],
            "Director" : response["metadata"]["Director"],
            "Genre" : response["metadata"]["Generes"],
            "year" : response["metadata"]["year"],
            "Rating" : response["metadata"]["Rating"],

        }
    )

  df = pd.DataFrame(response_data)
  return df


In [24]:
search("A time travel adventure",None,None,5)

,Title,Overview,Director,Genre,year,Rating
0,The Exotic Time Machine,A time machine transports two curious pleasure...,Felicia Sinclair,['Sci-Fi'],-1998,3.6
1,Timeline,A group of archaeologists become trapped in th...,Richard Donner,"['Action', 'Adventure', 'Sci-Fi']",-2003,5.6
2,Norman,A time traveler and his A.I. companion. Norman...,Joel Guelzo,"['Adventure', 'Sci-Fi', 'Thriller']",(IV) (2019),3.6
3,A Sound of Thunder,"A single mistake in the past, by a time travel...",Peter Hyams,"['Action', 'Adventure', 'Horror']",-2005,4.2
4,Aditya 369,A group of children use a time machine to inve...,Singeetam Srinivasa Rao,"['Adventure', 'Comedy', 'Sci-Fi']",-1991,8.5


In [25]:
import gradio as gr

# Define possible genres
genres = unique_generes.tolist()
iface = gr.Interface(
    fn=search,
    inputs=[
        gr.Textbox(lines=5, placeholder="Type here any search...", label="Consulta"),
        gr.Dropdown(choices=genres, label="Movie Genre"),
        gr.Slider(minimum=1, maximum=10, value=5, label="Minum Score"),
        gr.Number(minimum=1, maximum=10, value=3, label="Number of movies to search")

    ],
    outputs=gr.Dataframe(type="pandas", label="Resultados"),
    title="Movie Searcher",
    description="type the search select the genre and define the minnum rank to search movies.",
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2c81b46081a059abf8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
